# qwen-long 长上下文测试

很奇怪，加入内容后的对话，没有返回内容。

只有 `qwen-max-longcontext` 能正常返回，效果还不错

## 测试是否可用

In [1]:
%%bash

TOKEN=sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750
HOST_URL=http://192.168.0.72:3000 

curl -s "$HOST_URL"/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $TOKEN" \
  -d '{
     "model": "qwen-long",
     "messages": [{"role": "user", "content": "你谁啊"}],
     "temperature": 0.7
   }' | jq .

{
  "id": "f02ffc27-b694-9dad-a543-7a1a65227571",
  "model": "qwen",
  "object": "chat.completion",
  "created": 1717750507,
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "我是通义千问，由阿里云开发的AI助手。我可以回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 3,
    "completion_tokens": 33,
    "total_tokens": 36
  }
}


## 长文本测试

In [2]:
from openai import OpenAI

base_url = "http://192.168.0.72:3000/v1"
api_key = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"

client = OpenAI(api_key=api_key, base_url=base_url)

def process_stream(stream):
    for chunk in stream:
        if chunk.usage:
            print()
            print(chunk.usage)
        else:
            print(chunk.choices[0].delta.content or "", end="")

In [11]:
%%time

stream = client.chat.completions.create(
    model="qwen-long",
    messages=[{"role": "user", "content": "你谁啊？"}],
    stream=True,
    stream_options= {"include_usage": True}
)

process_stream(stream)

我是通义千问，由阿里云开发的AI助手。我可以回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？CPU times: user 14 ms, sys: 3.82 ms, total: 17.8 ms
Wall time: 3.42 s


In [16]:
%%time

file_path = './content-18k.txt'

# Read the content of the file and store it in a string variable
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

len(content)

CPU times: user 839 µs, sys: 89 µs, total: 928 µs
Wall time: 851 µs


17587

In [19]:
%%time

stream = client.chat.completions.create(
  model="qwen-max-longcontext",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，详细归纳一下，形成要点列表。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

文章要点归纳如下：

1. **大争之世的定义与背景**
   - “大争之世”意味着全球秩序的根本性转变，从西方主导转向“东升西降”。
   - 这一转变体现在国际格局、现代化模式、工业革命和全球治理的根本变化上。
   - 中美结构性冲突是“大争之世”的一个显著特征，源于中国崛起与美国维护霸权的矛盾。

2. **国际格局的变化**
   - 新的国际格局特点是“东升西降”，中国、印度和东盟等东方力量崛起，而欧洲和日本相对衰落。
   - 西方主导的国际格局正在解体，东方国家在经济和人口规模上占据更大比重。

3. **现代化模式的转变**
   - 中国式现代化与西方模式不同，强调社会主义特色，注重全面、协调和和平发展。
   - 中国现代化的成功挑战了西方模式的唯一性，提供了新的发展路径。

4. **新的工业革命**
   - 第四次工业革命中，中国在人工智能、生命科学、新材料和新能源技术等领域扮演关键角色。
   - 中国在某些领域实现领先，为人类文明进步作出贡献，打破了西方在工业革命中的垄断。

5. **全球治理的新模式**
   - 面对全球安全和发展的挑战，中国提出全球安全倡议和“一带一路”等方案，推动全球治理改革。
   - 强调公平、合作与可持续性，反对西方中心主义，为全球治理提供中国智慧和方案。

6. **中国作为关键变量的角色**
   - 中国凭借工业化、人口与国土规模优势、文化精神和和平发展策略成为“东升”的核心力量。
   - 与美国的结构性矛盾基于利益和世界观差异，中国需内部团结、经济发展、斗争精神和积极外交以应对。

文章整体分析了国际秩序变迁中中国崛起的背景、影响及中美关系，以及中国在新国际格局、现代化、工业革命和全球治理中的作用，强调了中国作为“大争之世”关键变量的地位与应对策略。CPU times: user 58.2 ms, sys: 15.8 ms, total: 74.1 ms
Wall time: 38.4 s


In [20]:
%%time

stream = client.chat.completions.create(
  model="qwen-max",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，详细归纳一下，形成要点列表。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

CPU times: user 9.21 ms, sys: 84 µs, total: 9.29 ms
Wall time: 579 ms
